In [1]:
from pyspark.sql import SparkSession, functions as f

In [2]:
spark = (
    SparkSession.builder.appName("Preprocessing_Yellow")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '3g')   
    .config('spark.executor.memory', '4g')  
    .config('spark.executor.instances', '2')  
    .config('spark.executor.cores', '2')
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
23/09/27 16:49:23 WARN Utils: Your hostname, DulanComputer resolves to a loopback address: 127.0.1.1; using 172.30.15.25 instead (on interface eth0)
23/09/27 16:49:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/27 16:49:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
consumers = spark.read.parquet("../../../data/insights/pre_insights/consumers.parquet/")

In [4]:
consumers_aggregated = consumers.groupBy("merchant_abn").agg(
                        f.countDistinct("consumer_id").alias("number_of_unique_consumers"),
                        f.avg("consumer_fraud_probability_%").alias("average_consumer_fraud_probability"))

In [5]:
consumers_aggregated.filter(consumers_aggregated["merchant_abn"] == 71118957552).show()

+------------+--------------------------+----------------------------------+
|merchant_abn|number_of_unique_consumers|average_consumer_fraud_probability|
+------------+--------------------------+----------------------------------+
| 71118957552|                        37|                               0.0|
+------------+--------------------------+----------------------------------+



In [5]:
repeat_consumers = consumers.groupBy("merchant_abn", "consumer_id").agg(f.count("consumer_id").alias("consumer_order_times"))
repeat_consumers_order_times = repeat_consumers.groupBy("merchant_abn").agg(f.avg("consumer_order_times").alias("average_repeat_transactions_per_consumer"))
repeat_consumers = repeat_consumers.filter(repeat_consumers.consumer_order_times > 1)
repeat_consumers_count = repeat_consumers.groupBy("merchant_abn").agg(f.count("consumer_order_times").alias("number_of_repeat_consumers"))

In [ ]:
repeat_consumers_order_times.orderBy(f.col("average_repeat_transactions_per_consumer").desc()).show()

In [6]:
consumers_aggregated = consumers_aggregated.join(repeat_consumers_count, on = "merchant_abn", how = "inner")
consumers_aggregated = consumers_aggregated.join(repeat_consumers_order_times, on = "merchant_abn", how = "inner")

In [7]:
consumers_aggregated = consumers_aggregated.withColumn("consumer_retainability", consumers_aggregated.number_of_repeat_consumers/consumers_aggregated.number_of_unique_consumers)

In [ ]:
consumers_aggregated.orderBy(f.col("number_of_unique_consumers").desc()).show()

In [8]:
consumers_aggregated.write.mode("overwrite").parquet("../../../data/insights/agg_insight_data/consumers_agg.parquet")

In [9]:
spark.stop()